In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/home/lukasz/Portfolio/Dataset/synthetic_fraud_dataset.csv")
df.head()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,...,7,437.63,3,Amex,65,883.17,Biometric,0.8494,0,0
1,TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-07 04:01:00,75725.25,Mobile,New York,Clothing,0,...,13,478.76,4,Mastercard,186,2203.36,Password,0.0959,0,1
2,TXN_199,USER_2734,28.96,Online,2023-06-20 15:25:00,1588.96,Tablet,Mumbai,Restaurants,0,...,14,50.01,4,Visa,226,1909.29,Biometric,0.8400,0,1
3,TXN_12447,USER_2617,254.32,ATM Withdrawal,2023-12-07 00:31:00,76807.20,Tablet,New York,Clothing,0,...,8,182.48,4,Visa,76,1311.86,OTP,0.7935,0,1
4,TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44:00,92354.66,Mobile,Mumbai,Electronics,0,...,14,328.69,4,Mastercard,140,966.98,Password,0.3819,1,1


In [4]:
print(df.Timestamp)  # sprawdź dokładną nazwę
print(df['Timestamp'].dtype)  # sprawdź typ danych tej kolumny
print(df['Timestamp'].head()) 

0        2023-08-14 19:30:00
1        2023-06-07 04:01:00
2        2023-06-20 15:25:00
3        2023-12-07 00:31:00
4        2023-11-11 23:44:00
                ...         
49995    2023-01-29 18:38:00
49996    2023-05-09 08:55:00
49997    2023-01-30 19:32:00
49998    2023-03-09 19:47:00
49999    2023-08-19 23:57:00
Name: Timestamp, Length: 50000, dtype: object
object
0    2023-08-14 19:30:00
1    2023-06-07 04:01:00
2    2023-06-20 15:25:00
3    2023-12-07 00:31:00
4    2023-11-11 23:44:00
Name: Timestamp, dtype: object


In [5]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

In [11]:
print(df['Timestamp'].dtype) 

datetime64[ns]


In [13]:
def analyze_date_columns(df, results, date_column, check_levels=['months', 'days', 'hours', 'minutes', 'seconds']):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    print("Typ kolumny:", df[date_column].dtype)
    print("Czy kolumna istnieje?", date_column in df.columns)
    results.append(f"\n### Analiza kolumny datowej: {date_column} ###")    
    if date_column not in df.columns or not np.issubdtype(df[date_column].dtype, np.datetime64):
        results.append(f"- {date_column}: Nie podano kolumny lub podana kolumna nie istanieje (może nie poprawny typ danych)")
        return
    min_date = df[date_column].min()
    max_date = df[date_column].max()
    print(min_date)
    if pd.isnull(min_date) or pd.isnull(max_date):
        results.append(f"- {date_column}: Brak prawidłowych wartości datowych.")
        return
    results.append(f"- Zakres dat: od {min_date} do {max_date}")
    for level in check_levels:
        freq = {'months': 'M', 'days': 'D', 'hours': 'H', 'minutes': 'T', 'seconds': 'S'}.get(level)
        if freq:
            full_range = pd.date_range(start=min_date, end=max_date, freq=freq)
            existing_values = df[date_column].dropna().dt.to_period(freq)
            missing_values = sorted(set(full_range.to_period(freq)) - set(existing_values))
            results.append(f"  - Braki dla {level}: {len(missing_values)}")
            # Dodanie przykładowych brakujących wartości (max 5)
            if missing_values:
                example_missing = ', '.join(str(v) for v in missing_values[:5])
                results.append(f"    - Przykłady brakujących wartości: {example_missing}")

In [16]:
results = []
analyze_date_columns(df,results,'Timestamp',('H','S'))

Typ kolumny: datetime64[ns]
Czy kolumna istnieje? True
2023-01-01 00:01:00


In [17]:
results

['\n### Analiza kolumny datowej: Timestamp ###',
 '- Zakres dat: od 2023-01-01 00:01:00 do 2023-12-31 23:50:00']

In [2]:
def draw_tree(levels):
    """
    Rysuje drzewo z '#' z podaną liczbą poziomów.

    :param levels: Liczba poziomów drzewa.
    """
    for i in range(1, levels + 1):
        spaces = " " * (levels - i)
        hashes = "#" * (2 * i - 1)
        print(spaces + hashes )

    # Pień drzewa
    trunk_width = levels // 3 if levels > 3 else 1
    trunk_height = max(1, levels // 3)
    trunk_space = " " * (levels - trunk_width // 2 - 1)
    
    for _ in range(trunk_height):
        print(trunk_space + "#" * trunk_width)

# Przykład użycia
draw_tree(10)


         #
        ###
       #####
      #######
     #########
    ###########
   #############
  ###############
 #################
###################
        ###
        ###
        ###
